# Example Python Development using Neo4j

Click on the link below to open a Colab version of the notebook. You will be able to create your own version.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github.com/neo4j-field/neo4j-nz-bootcamp/blob/master/100_neo4j_python_development/neo4j_python_example.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo">Run your own notebook in Colab
    </a>
  </td>
</table>

## Install `neo4j` Python Library

In [ ]:
# Install Neo4j Python Client
import sys
!{sys.executable} -m pip install neo4j

## Import Library

In [1]:
from neo4j import GraphDatabase

import numpy as np
import pandas as pd
import json

## Neo4j Helper Function

In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

## Credentials

Please change the value of the password and database name which is your username

In [3]:
# Change URL
uri = "URL_HERE:7687"
user="neo4j"
# Change Password
pwd = "PW_HERE"
db="neo4j"

## Connect to Neo4j Database

In [4]:
conn = Neo4jConnection(uri=uri, user=user, pwd=pwd)

conn.query("MATCH (n) RETURN COUNT(n)", db=db)

[<Record COUNT(n)=1044>]

## Clean Database

In [5]:
# query = '''
# MATCH(n) DETACH DELETE n
# '''
# result = conn.query(query, db=db)

## Create node data

In [16]:
pdf_categories = pd.read_csv("https://raw.githubusercontent.com/adam-cowley/northwind-neo4j/master/data/categories.csv")

In [17]:
pdf_categories.head()

,categoryID,categoryName,description,picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an...",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
2,3,Confections,"Desserts, candies, and sweet breads",0x151C2F00020000000D000E0014002100FFFFFFFF4269...
3,4,Dairy Products,Cheeses,0x151C2F00020000000D000E0014002100FFFFFFFF4269...
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",0x151C2F00020000000D000E0014002100FFFFFFFF4269...


In [25]:
query = '''
UNWIND $rows AS row
MERGE (c:Categories2 {categoryID: toString(row.categoryID)})
ON CREATE SET c.categoryName = row.categoryName
'''
result = conn.query(query, parameters={'rows':pdf_categories.to_dict('records')}, db=db)

## Connect Categories2 and Category

In [27]:
query = '''
UNWIND $rows AS row
MATCH (c1:Categories2 {categoryID: toString(row.categoryID)})
MATCH (c2:Category {categoryID: toString(row.categoryID)})
MERGE (c1)-[:SAME_AS]->(c2);
'''
result = conn.query(query, parameters={'rows':pdf_categories.to_dict('records')}, db=db)

## Data Retrieval

In [28]:
query = '''
MATCH (c:Customer {customerID: $customerId})
RETURN c;
'''
result = conn.query(query, parameters={"customerId": "ANTON"}, db=db)

In [29]:
df_result = pd.DataFrame([dict(record["c"]) for record in result])
df_result

,phone,companyName,customerID,fax
0,(5) 555-3932,Antonio Moreno Taquería,ANTON,NULL
